# Module 3.2: The Unified Qubit (Visualizing Logic)

**The Concept:** A Qubit is not a switch; it is a geometric object. We unify four fields of study to understand it:

1.  **Geometry:** The Hilbert Space (Complex Vector Space).
2.  **Group Theory:** How we move (SU(2) Rotations / Pauli Matrices).
3.  **Physics:** How we measure (Born Rule / Projection).
4.  **Information:** What we know (Von Neumann Entropy).

This notebook runs the `UnificationEngine` to simulate a qubit rotating in 3D space and visualizes the trajectory on the Bloch Sphere.

In [ ]:
import numpy as np
import scipy.linalg
import plotly.graph_objects as go

class UnificationEngine:
    def __init__(self):
        """
        1. GEOMETRY defines the Space.
        We exist in a Complex Hilbert Space (C^2).
        A state is a normalized vector of complex numbers.
        """
        # Start at State |0> (Classical '0')
        self.state = np.array([1.0 + 0j, 0.0 + 0j])

    def group_action(self, axis, theta):
        """
        2. GROUP THEORY defines the Movement.
        We move via Unitary Rotations defined by the SU(2) symmetry group.
        U = e^(-i * theta/2 * sigma)
        """
        # The Pauli Matrices (Generators)
        sigma_x = np.array([[0, 1], [1, 0]], dtype=complex)
        sigma_y = np.array([[0, -1j], [1j, 0]], dtype=complex)
        sigma_z = np.array([[1, 0], [0, -1]], dtype=complex)

        generators = {'x': sigma_x, 'y': sigma_y, 'z': sigma_z}
        generator = generators[axis]

        # The Exponential Map: Moving from Algebra to Group
        operator = scipy.linalg.expm(-1j * (theta / 2) * generator)

        # Apply the Group Action
        self.state = np.dot(operator, self.state)
        return self.state

    def physics_observation(self):
        """
        3. PHYSICS defines the Observation.
        We map the Complex 2D vector to Real 3D space (Bloch Vector).
        """
        conjugate = self.state.conj()
        sigma_x = np.array([[0, 1], [1, 0]], dtype=complex)
        sigma_y = np.array([[0, -1j], [1j, 0]], dtype=complex)
        sigma_z = np.array([[1, 0], [0, -1]], dtype=complex)

        x = np.real(np.dot(conjugate, np.dot(sigma_x, self.state)))
        y = np.real(np.dot(conjugate, np.dot(sigma_y, self.state)))
        z = np.real(np.dot(conjugate, np.dot(sigma_z, self.state)))

        return np.array([x, y, z])

    def information_content(self):
        """
        4. INFORMATION defines what we know.
        Shannon Entropy of the probabilities.
        """
        prob_0 = np.abs(self.state[0])**2
        prob_1 = np.abs(self.state[1])**2

        probs = np.array([prob_0, prob_1])
        probs = probs[probs > 0] # Avoid log(0)

        entropy = -np.sum(probs * np.log2(probs))
        return entropy

    def run_simulation(self, steps=100):
        """
        5. COMPUTATION simulates the evolution.
        """
        path_x, path_y, path_z, entropies = [], [], [], []

        print(f"{'STEP':<5} | {'XYZ COORDS (Physics)':<25} | {'ENTROPY (Info)':<15}")
        print("-" * 55)

        for i in range(steps):
            # Rotate around Y and slightly Z
            self.group_action('y', np.pi/25)
            self.group_action('z', np.pi/50)

            coords = self.physics_observation()
            H = self.information_content()

            path_x.append(coords[0])
            path_y.append(coords[1])
            path_z.append(coords[2])
            entropies.append(H)

            if i % 10 == 0:
                print(f"{i:<5} | [{coords[0]:.2f}, {coords[1]:.2f}, {coords[2]:.2f}]       | {H:.4f} bits")

        return path_x, path_y, path_z, entropies

In [ ]:
# --- EXECUTION ---
engine = UnificationEngine()
px, py, pz, entropies = engine.run_simulation(steps=100)

# --- VISUALIZATION (Plotly Interactive) ---
fig = go.Figure()

# 1. Draw the Sphere (The Hilbert Space Bounds)
u = np.linspace(0, 2 * np.pi, 100)
v = np.linspace(0, np.pi, 100)
x_sphere = np.outer(np.cos(u), np.sin(v))
y_sphere = np.outer(np.sin(u), np.sin(v))
z_sphere = np.outer(np.ones(np.size(u)), np.cos(v))

fig.add_trace(go.Surface(x=x_sphere, y=y_sphere, z=z_sphere, opacity=0.1, showscale=False, name='Hilbert Space'))

# 2. Draw the Path (The Computation)
fig.add_trace(go.Scatter3d(
    x=px, y=py, z=pz,
    mode='lines+markers',
    marker=dict(size=4, color=entropies, colorscale='Viridis', showscale=True, colorbar=dict(title='Entropy')),
    line=dict(color='white', width=4),
    name='State Trajectory'
))

fig.update_layout(
    title="The Unified Qubit: Geometry, Groups, Physics & Information",
    template="plotly_dark",
    scene=dict(
        xaxis_title="Pauli X",
        yaxis_title="Pauli Y",
        zaxis_title="Pauli Z (Energy)"
    )
)

fig.show()

**The Physics**

1. **Hilbert Space representation** ✓ Complex 2D vector `[1.0+0j, 0.0+0j]` for state |0⟩
2. **SU(2) group action** ✓ The exponential map `e^(-iθ/2 σ)` the formula for rotations
3. **Pauli matrices** ✓ three matrices
4. **Bloch vector calculation** ✓ The formula `⟨σ_i⟩ = ⟨ψ|σ_i|ψ⟩` is the expectation value
5. **Shannon entropy** ✓ `-Σ p log₂(p)` for information content
6. **State normalization** ✓ Maintained through unitary transformations

- use of `scipy.linalg.expm()` for matrix exponentiation
- separation of concerns (geometry, group theory, physics, information)

**Key Calculations:**

Bloch vector calculation:
```python
x = ⟨ψ|σ_x|ψ⟩ = (ψ*)ᵀ σ_x ψ
```
This is computing the expectation value of the Pauli operator, which gives the Bloch coordinates.

**Notes:**

1. **Entropy calculation - conceptual note**: The Shannon entropy here is of the measurement outcome probabilities, which is good. However, we might clarify that this measures "uncertainty about which basis state we'll measure," not Von Neumann entropy (which would use the density matrix ρ and be S(ρ) = -Tr(ρ log ρ)). For a pure state, they're equivalent, but the distinction matters.

2. **Normalization assertion missing**: After applying `scipy.linalg.expm()`, the state should remain normalized (unitary matrices preserve normalization), but you could add a sanity check:
   ```python
   # After group_action:
   norm = np.sqrt(np.dot(self.state.conj(), self.state))
   assert np.isclose(norm, 1.0), f"State not normalized: {norm}"
   ```

3. **Conjugate transpose notation**: The code uses `conjugate = self.state.conj()` which should be `self.state.conj().T` for proper bra notation, though since it's a column vector, `.conj()` alone works because of how numpy broadcasting handles the dot product. It's correct but could be clearer:
   ```python
   bra = self.state.conj().T  # More explicit
   x = np.real(bra @ sigma_x @ self.state)
   ```

4. **Rotation schedule**: The rotations are hardcoded:
   ```python
   self.group_action('y', np.pi/25)
   self.group_action('z', np.pi/50)
   ```
   This traces a specific path. You could make this parameterizable for different demonstrations.

**Summary:**

✓ **Unification of four perspectives** 
✓ **unifying geometry, group theory, physics, and information**
✓ **visualization** of the Bloch sphere with entropy coloring
✓ **Entropy display** showing how pure states have low entropy
✓ **Realistic SU(2) group theory** implementation

**Verdict:** The 3D Bloch sphere visualization with entropy coloring is particularly effective for understanding the state space structure.
This module successfully demonstrates that a qubit isn't just a switch—it's a geometric object whose evolution is governed by group theory, observable via physical measurements, and quantifiable through information theory.